![Xilinx Logo](images/xilinx_logo.png "Xilinx Logo")

# 1. Introduction

This notebook demonstrates how to create two parallel video pipelines using the GStreamer multimedia framework:
* The first pipeline captures video from a V4L2 device and displays the output on a monitor using a DRM/KMS display device.
* The second pipeline decodes a VP9 encoded video file and displays the output on the same monitor using the same DRM/KMS display device.

The display device contains a video mixer which allows targeting different video planes for the individual pipelines with  programmable x/y-offsets as well as width and height.

Refer to:
* nb1 for more details on the video file decode pipeline
* nb2 for more details on the V4L2 capture pipeline
* nb3 for more details on the video mixer configuration and display pipeline

In this notebook, you will:
1. Create two parallel GStreamer video pipelines using the ``parse_launch()`` API
2. Create a GStreamer pipeline graph and view it inside this notebook.

# 2. Imports and Initialization

Import all python modules required for this notebook. 

In [ ]:
from IPython.display import Image, display, clear_output
import glob
import subprocess
import pydot
import sys
import gi
gi.require_version('Gst', '1.0')
from gi.repository import GObject, GLib, Gst

This is the Base TRD notebook 4 (nb4).

In [ ]:
nb = "nb4"

Create a directory for saving the pipeline graph as dot file. Set the GStreamer debug dot directory environment variable to point to that directory.

In [ ]:
dotdir = "/home/root/gst-dot/" + nb
!mkdir -p $dotdir
%env GST_DEBUG_DUMP_DOT_DIR = $dotdir

Initialize the GStreamer library. Enable debug by setting the debug string, set default to level 1 for all categories.

In [ ]:
Gst.init(None)
Gst.debug_set_threshold_from_string('*:1', True)

# 3. Create String Representation of the First GStreamer Pipeline

The first pipeline consist of the following elements:
* ``mediasrcbin``
* ``caps``
* ``kmssink``

The ``get_media_by_device`` function returns the matching media node for a given video capture source. The following sources are supported in this notebook:
* ``vivid`` : virtual video device (default)
* ``usb`` : requires USB webcam
* ``mipi`` : platform1 only, requires FMC card
* ``hdmi`` : platform3 only, requires HDMI input

In [ ]:
def get_media_dev_by_name(src):
    sources = {
        'vivid' : 'vivid',
        "usb" : 'uvcvideo',
        'mipi' : 'vcap_csi',
        'hdmi' : 'vcap_hdmi'
    }
    devices = glob.glob('/dev/media*')
    for dev in devices:
        proc = subprocess.run(['media-ctl', '-d', dev, '-p'], capture_output=True, encoding='utf8')
        for line in proc.stdout.splitlines():
            if sources[src] in line:
                return dev

Describe the ``mediasrcbin`` element and its properties as string representation.

In [ ]:
source = "vivid" # Change source to vivid, usb, mipi, hdmi

media_device = get_media_dev_by_name(source) 
if media_device is None:
    raise Exception('Unable to find video source ' + source + '. Make sure the device is plugged in, powered, and the correct platform is used.')

io_mode = "mmap"
if source == "mipi" or source == "hdmi":
    io_mode = "dmabuf"

src_1 = "mediasrcbin media-device=" + media_device + " v4l2src0::io-mode=" + io_mode

Describe the ``caps`` filter element as string representation.

In [ ]:
width = 1280
height = 720
fmt = "YUY2"

caps = "video/x-raw, width=" + str(width) + ", height=" + str(height) + ", format=" + fmt
if source == "mipi" or source == "hdmi":
    fps = "60/1"
    caps = caps + ", framerate=" + fps

Describe the ``kmssink`` element and its properties as string representation.

In [ ]:
plane_id_1 = 38
xoff_1 = 0
yoff_1 = 0
render_rectangle_1 = "<" + str(xoff_1) + "," + str(yoff_1) + "," + str(width) + "," + str(height) + ">"

sink_1 = "kmssink" + " plane-id=" + str(plane_id_1) + " render-rectangle=" + render_rectangle_1

Create a string representation of the first pipeline by concatenating the individual element strings.

In [ ]:
pipe_1 = src_1 + " ! " + caps + " ! " + sink_1
print(pipe_1)

# 4. Create String Representation of the Second GStreamer Pipeline

The second pipeline consist of the following elements:
* ``multifilesrc``
* ``decodebin``
* ``videoconvert``
* ``kmssink``

Describe the ``multifilesrc`` element and its properties as string representation.

In [ ]:
file_name = "/usr/share/movies/Big_Buck_Bunny_4K.webm.360p.vp9.webm"
loop = True

src_2 = "multifilesrc location=" + file_name + " loop=" + str(loop)

Describe the ``decodebin`` and ``videoconvert`` elements as string representations.

In [ ]:
dec = "decodebin"
cvt = "videoconvert"

Describe the ``kmssink`` element and its properties as string representation.

**Note:** The same ``kmssink`` element and ``driver-name`` property are used as in pipeline 1, only the ``plane-id`` and the ``render-rectangle`` properties are set differently. The output of this pipeline is shown on a different plane and the x/y-offsets are set such that the planes of pipeline 1 and 2 don't overlap.

In [ ]:
plane_id_2 = 39
xoff_2 = 0
yoff_2 = 720
width_2 = 640
height_2 = 360
render_rectangle_2 = "<" + str(xoff_2) + "," + str(yoff_2) + "," + str(width_2) + "," + str(height_2) + ">"

sink_2 = "kmssink" + " plane-id=" + str(plane_id_2) + " render-rectangle=" + render_rectangle_2

Create a string representation of the second pipeline by concatenating the individual element strings.

In [ ]:
pipe_2 = src_2 + " ! " + dec + " ! " + cvt + " ! "+ sink_2
print(pipe_2)

# 5. Create and Run the GStreamer Pipelines

Parse the string representations of the first and second pipeline as a single pipeline graph.

In [ ]:
pipeline = Gst.parse_launch(pipe_1 + " " + pipe_2)

The ``bus_call`` function listens on the bus for ``EOS`` and ``ERROR`` events. If any of these events occur, stop the pipeline (set to ``NULL`` state) and quit the main loop.

In case of an ``ERROR`` event, parse and print the error message.

In [ ]:
def bus_call(bus, message, loop):
    t = message.type
    if t == Gst.MessageType.EOS:
        sys.stdout.write("End-of-stream\n")
        pipeline.set_state(Gst.State.NULL)
        loop.quit()
    elif t == Gst.MessageType.ERROR:
        err, debug = message.parse_error()
        sys.stderr.write("Error: %s: %s\n" % (err, debug))
        pipeline.set_state(Gst.State.NULL)
        loop.quit()
    return True

Start the pipeline (set to ``PLAYING`` state), create the main loop and listen to messages on the bus. Register the ``bus_call`` callback function with the ``message`` signal of the bus. Start the main loop.

The video will be displayed on the monitor. 

To stop the pipeline, click the square shaped icon labelled 'Interrupt the kernel' in the top menu bar. Create a dot graph of the pipeline topology before stopping the pipeline. Quit the main loop.

In [ ]:
pipeline.set_state(Gst.State.PLAYING);

loop = GLib.MainLoop()
bus = pipeline.get_bus()
bus.add_signal_watch()
bus.connect("message", bus_call, loop)

try:
    loop.run()
except:
    sys.stdout.write("Interrupt caught\n")
    Gst.debug_bin_to_dot_file(pipeline, Gst.DebugGraphDetails.ALL, nb)
    pipeline.set_state(Gst.State.NULL)
    loop.quit()
    pass

# 6. View the Pipeline dot Graph

Register dot plugins for png export to work.

In [ ]:
!dot -c

Convert the dot file to png and display the pipeline graph. The image will be displayed below the following code cell. Double click on the generate image file to zoom in.

**Note:** This step may take a few seconds. Also, compared to previous notebooks, two disjoint graphs are displayed in the same image as we have created two parallel pipelines in this example.

In [ ]:
dotfile = dotdir + "/" + nb + ".dot"
graph = pydot.graph_from_dot_file(dotfile, 'utf-8')
display(Image(graph[0].create(None, 'png', 'utf-8')))

# 7. Summary

In this notebook you learned how to:
1. Create two parallel GStreamer pipelines from a string representation using the ``parse_launch()`` API
2. Export the pipeline topology as a dot file image and display it in the notebook

<center>Copyright© 2019 Xilinx</center>